In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc

import matplotlib.pyplot as plt
import seaborn as sns

from anticor_features.anticor_features import get_anti_cor_genes
import topo as tp
from topo.tpgraph.intrinsic_dim import IntrinsicDim

In [ ]:
import datetime
print(datetime.datetime.now())

2025-02-28 15:27:12.092313


In [ ]:
adata = sc.read_h5ad('/Users/npapadop/Documents/teaching/2023/advanced_scRNAseq/data/clytia/cuff_adata_unfilt.h5ad')

cells = pd.read_csv('/Users/npapadop/Documents/teaching/2023/advanced_scRNAseq/data/clytia/meta.tsv', sep='\t', index_col=0, header=0)

adata = adata[cells.index].copy()
adata.obs = adata.obs.join(cells.drop(columns=['batch']))

In [ ]:
genes = pd.read_csv('/Users/npapadop/Documents/data/metazoan_atlases/clytia_hemisphaerica/h_sap_nr-clytia_full_rna.txt', sep='\t', header=None, skiprows=4)
genes.columns = ['Protein 1', 'Protein 2', 'Protein ID1', 'Protein ID2', 'Orthology type', 'OMA group']

In [ ]:
genes['clytia'] = genes['Protein ID2'].str.split(' ').str[0].str.split('|').str[1]
genes['symbol'] = genes['Protein ID1'].str.split(' ').str[0].str.split('|').str[1]
genes['description'] = genes['Protein ID1'].str.split('|').str[9]

In [ ]:
genes['clytia'].value_counts()

clytia
XLOC_006108    28
XLOC_039998    20
XLOC_036708    19
XLOC_040069    19
XLOC_019521    17
               ..
XLOC_044282     1
XLOC_008180     1
XLOC_004816     1
XLOC_041601     1
XLOC_035288     1
Name: count, Length: 4944, dtype: int64

In [ ]:
reduced = genes.groupby('clytia', group_keys=False).apply(lambda x: x.sample(1), include_groups=True)

/var/folders/md/d6lwwbv97xb6g6ddypntnprh0000gp/T/ipykernel_7123/3535290506.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reduced = genes.groupby('clytia', group_keys=False).apply(lambda x: x.sample(1), include_groups=True)


In [ ]:
adata.var = adata.var.join(reduced.set_index('clytia')[['symbol', 'description']])

In [ ]:
adata.write_h5ad('/Users/npapadop/Documents/data/metazoan_atlases/clytia_hemisphaerica/counts_and_metadata.h5ad')